# **Trabalho 1 - Introdução ao Aprendizado de Máquina (EEL891)**
### Classificação: sistema de apoio à decisão p/ aprovação de crédito

---

**Aluno:** Danilo Davi Gomes Fróes

**DRE:** 124026825

**Usuário Kaggle:** *dgfroes*

**Semestre:** 2025.01

---

### **1. Introdução**

O objetivo principal é construir um modelo classificador que, a partir dos dados de um solicitante, seja capaz de predizer com melhor acurácia a probabilidade de ele se tornar inadimplente (variável-alvo `inadimplente`). A finalidade prática de tal modelo é apoiar a decisão de aprovação de crédito, minimizando o risco para a instituição financeira.

O trabalho foi desenvolvido usando conhecimentos aprendidos durante as aulas e também do meu projeto de iniciação científica no Laboratório de Simulação e Otimização de Sistemas Produtivos (LASOS) do Instituto Nacional de Tecnologia (INT). O projeto é para o desenvolvimento de uma biblioteca em python de machine learning para otimizar a produção de múltiplas pipelines e gerar insights úteis de retorno para o usuário sem a necessidade de construção de grandes códigos.
Deixo aqui um resumo recente do trabalho desenvolvido até então, que futuramente será lançada publicamente para uso da comunidade do python.

[Resumo - LasosLibrary](https://docs.google.com/document/d/1T4g_4vvwjcM2SF-lBHUC7RIizCHBxomc/edit?usp=sharing&ouid=107178103419237221884&rtpof=true&sd=true)

In [ ]:
# Bibliotecas de Manipulação de Dados e Visualização
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Pré-processamento e Validação
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
import optuna

# Métricas
from sklearn.metrics import accuracy_score, make_scorer

# Remoção de avisos para evitar poluição visual
import warnings
warnings.filterwarnings('ignore')

# Configurações de Visualização
sns.set_style('whitegrid')

### **2. Análise Exploratória de Dados (EDA - Exploratory Data Analysis)**
Uma parte essencial para treinar modelos de machine learning é entender o que são os dados que estamos utilizando, dessa forma, fiz uma abordagem utilizando funções da biblioteca pandas e o dicionário de dados disponibilizado para obter informações essenciais para o desenvolvimento do projeto.

- Nas células abaixo, realizo a obtenção dos dados em CSV como `DataFrames` do pandas e utilizo as funções `info`, `describe` e `value_counts` para uma análise mais aprofundada de cada feature e do balanceamento da target.

In [ ]:
df_treino_raw = pd.read_csv('conjunto_de_treinamento.csv')
df_teste_raw = pd.read_csv('conjunto_de_teste.csv')

print(f"Info treinamento: {df_treino_raw.info()}")
display(df_treino_raw.describe())
print(f"Target treinamento: {df_treino_raw['inadimplente'].value_counts()}")

print(f"Info teste: {df_teste_raw.info()}")
display(df_teste_raw.describe())

### **3. Preparação Inicial e Limpeza dos Dados**

A primeira etapa consistiu em estruturar e limpar os dados brutos. O objetivo aqui foi criar uma base de trabalho limpa e consistente. Foram testados diversos tipos de novas features ou manipulação das que existem, porém prejudicava o modelo.

#### 3.1. Definição das Matrizes de Features (X) e Vetor Alvo (y)

Primeiramente, eu separei a variável-alvo, `inadimplente`, do restante do conjunto de treino, criando o vetor `y`. As demais colunas formaram a matriz de features inicial, `X`.

Eu também realizei uma remoção seletiva de colunas com base no dicionário de dados e em uma análise inicial:
* `id_solicitante`: Removido por ser um identificador único, sem poder preditivo.
* `inadimplente`: Removido da matriz `X` por ser a variável-alvo.
* `possui_telefone_celular`: Removido pois, conforme o dicionário de dados, esta coluna possuía variância nula (todos os valores eram "N"), não contendo informação útil para o modelo.

Algumas colunas foram mantidas, mesmo que recomendadas a serem retiradas, pois mudavam o desempenho.

#### 3.2. Tratamento de Valores Ausentes Ocultos

Durante a inspeção dos dados, notei que muitos valores ausentes não estavam no formato padrão `NaN` (Not a Number), mas sim representados por strings como `''`, `'XX'` ou `' '`. Estes são "nulos ocultos" que não seriam detectados automaticamente pelas ferramentas de imputação.

Para corrigir isso, eu substituí todas as ocorrências dessas strings por `np.nan`. Essa padronização é um passo de limpeza fundamental, garantindo que todo e qualquer dado faltante seja corretamente reconhecido e tratado na etapa de pré-processamento do pipeline.

#### 3.3. Alinhamento dos Conjuntos de Treino e Teste

Para garantir a consistência e evitar erros durante a predição, é essencial que os dataframes de treino e teste tenham exatamente a mesma estrutura de colunas. A linha `X_teste = X_teste[X.columns]` assegura esse alinhamento, reordenando as colunas do conjunto de teste para que correspondam perfeitamente às do conjunto de treino.

In [ ]:
# Guardar IDs e alvo
ids_teste = df_teste_raw['id_solicitante']
y = df_treino_raw['inadimplente']

# Remover colunas com baixa variância ou redundantes
colunas_a_remover = ['id_solicitante', 'inadimplente', 'possui_telefone_celular']
X = df_treino_raw.drop(columns=colunas_a_remover, errors='ignore')
X_teste = df_teste_raw.drop(columns=colunas_a_remover, errors='ignore')

# Limpeza de nulos ocultos
nulos_ocultos = [' ', 'N/A', '', '?', 'XX', 'NULL']
X.replace(nulos_ocultos, np.nan, inplace=True)
X_teste.replace(nulos_ocultos, np.nan, inplace=True)

# Garantir consistência das colunas
X_teste = X_teste[X.columns]

print(f"Número de features: {X.shape[1]}")

### **4. Pipeline de Pré-processamento e Estratégia de Validação**

Para garantir que o processo de transformação dos dados fosse robusto, reprodutível e livre de vazamento de dados (*data leakage*), eu construí um pipeline de pré-processing unificado utilizando o `ColumnTransformer` do Scikit-learn. Em paralelo, defini uma estratégia de validação cruzada para assegurar uma avaliação fidedigna da performance dos modelos.

#### 4.1. Tratamento de Features Numéricas

Para as colunas numéricas, eu criei um pipeline com dois passos essenciais:

1.  **Imputação com Mediana (`SimpleImputer`):** Para tratar os valores ausentes (`NaN`), optei por preenchê-los com a **mediana** da respectiva coluna. A mediana é uma medida de tendência central mais robusta a outliers do que a média, sendo a escolha ideal para dados financeiros e demográficos, que frequentemente possuem distribuições assimétricas.
2.  **Escalonamento Padrão (`StandardScaler`):** Após a imputação, eu padronizei as features numéricas. O `StandardScaler` transforma os dados para que tenham média (μ) igual a 0 e desvio padrão (σ) igual a 1. Este passo é crucial para o bom desempenho de muitos algoritmos, incluindo a Regressão Logística (que usei como meta-modelo no ensemble), pois garante que nenhuma feature domine o processo de aprendizado apenas por ter uma escala de magnitude maior que as outras.

#### 4.2. Tratamento de Features Categóricas

Para as colunas categóricas (que contêm texto ou códigos), o pipeline também seguiu dois passos:

1.  **Imputação com Moda (`SimpleImputer`):** Os valores ausentes foram preenchidos com a categoria mais frequente ("moda") da coluna. Esta é a abordagem padrão e mais lógica para dados categóricos.
2.  **Encoding com `OneHotEncoder`:** Como os modelos de machine learning requerem input numérico, eu converti as categorias em um formato que eles pudessem entender. Optei pelo **One-Hot Encoding**, que cria uma nova coluna binária (0 ou 1) para cada categoria única. Escolhi este método para evitar a criação de uma falsa relação ordinal que outros encoders (como o `LabelEncoder`) poderiam introduzir. O parâmetro `handle_unknown='ignore'` foi fundamental para que o modelo lidasse de forma elegante com categorias que pudessem aparecer no conjunto de teste sem terem sido vistas no treino.

#### 4.3. Estratégia de Validação Cruzada

Para avaliar os modelos, eu utilizei a **Validação Cruzada Estratificada (`StratifiedKFold`)** com 10 folds. A estratificação garante que a proporção de inadimplentes e bons pagadores (as classes da variável-alvo) seja a mesma em cada um dos 10 folds, espelhando a distribuição do dataset original. Isso é essencial em problemas de classificação para evitar que um fold tenha, por acaso, uma concentração muito diferente das classes, o que levaria a uma estimativa de performance instável e pouco confiável.

In [ ]:
# Identifica colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

# Pipeline para features numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para features categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combina os pipelines no ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Define a estratégia de validação cruzada
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

### **5. Otimização de Hiperparâmetros**

Com a estrutura de dados e o pipeline de pré-processamento definidos, o passo seguinte foi focar em extrair o máximo de performance dos algoritmos de machine learning. Para isso, realizei um processo de **otimização de hiperparâmetros**, que consiste em encontrar a melhor combinação de configurações para cada modelo.

#### 5.1. A Escolha da Estratégia: Otimização Bayesiana com Optuna

Para realizar essa busca, eu optei por utilizar a biblioteca `Optuna`, que implementa a **Otimização Bayesiana**. Escolhi esta abordagem em detrimento de métodos mais simples como *Grid Search* (ineficiente para espaços de busca grandes) ou *Random Search* (eficiente, mas não-inteligente) pelas seguintes razões:

A Otimização Bayesiana é um método de busca sequencial e inteligente. Ela trata a otimização como um problema de probabilidade, construindo um modelo estatístico interno (surrogate model) que mapeia os hiperparâmetros aos scores de performance. A cada iteração, ela utiliza os resultados das tentativas anteriores para decidir qual a próxima combinação de parâmetros tem a maior probabilidade de gerar um resultado melhor. Esse processo equilibra de forma eficiente a **exploração** (testar novas áreas do espaço de busca) e a **explotação** (focar em áreas que já se mostraram promissoras), permitindo encontrar soluções de alta qualidade com um número menor de tentativas.

#### 5.2. O Processo de Otimização

Minha implementação foi estruturada em torno de uma função `objective` genérica, projetada para avaliar qualquer um dos modelos candidatos. Para cada modelo que eu desejava otimizar (`LGBMClassifier` e `XGBClassifier`), um "estudo" do Optuna foi executado:

1.  **Espaço de Busca:** Para cada modelo, eu defini um espaço de busca customizado, focando nos seus hiperparâmetros mais influentes, como `n_estimators`, `learning_rate`, `max_depth`, e parâmetros de regularização (`reg_alpha`, `reg_lambda`, `gamma`), que são essenciais para controlar o *overfitting*.

2.  **Avaliação:** Em cada uma das 100 tentativas (`n_trials`), a função `objective` construiu um pipeline completo (pré-processador + modelo com os parâmetros da tentativa) e o avaliou utilizando nossa robusta estratégia de validação cruzada estratificada (`cv_strategy`).

3.  **Resultado:** A acurácia média dos 10 folds foi retornada ao `study` do Optuna, que utilizou essa informação para refinar sua busca. Ao final do processo, os melhores parâmetros encontrados para cada modelo foram armazenados para serem utilizados na construção do nosso ensemble final.

In [ ]:
# Dicionário para guardar os melhores parâmetros
best_params_dict = {}

def objective(trial, model_class):
    if model_class == LGBMClassifier:
        params = {
            'random_state': 42, 'n_jobs': -1, 'verbose': -1,
            'n_estimators': trial.suggest_int('n_estimators', 200, 1500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0, log=True),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        }
    elif model_class == XGBClassifier:
        params = {
            'random_state': 42, 'n_jobs': -1, 'eval_metric': 'logloss',
            'n_estimators': trial.suggest_int('n_estimators', 200, 1500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_int('gamma', 0, 5)
        }
    
    model = model_class(**params)
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    score = cross_val_score(pipeline, X, y, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
    return score.mean()

# Otimizando os modelos
for name, model_class in [("LGBM", LGBMClassifier), ("XGB", XGBClassifier)]:
    print(f"\nOtimizando {name}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_class), n_trials=100)
    best_params_dict[name] = study.best_params
    print(f"Melhor Acurácia para {name}: {study.best_value:.4f}")

### **6. Modelo Final: Ensemble com StackingClassifier**

Após o processo de otimização, eu possuía duas versões altamente ajustadas dos modelos mais promissores: LightGBM e XGBoost. A etapa final consistiu em combinar a força preditiva de ambos em um único modelo superior, utilizando uma técnica de ensemble avançada conhecida como **Stacking** (ou empilhamento).

#### 6.1. A Estratégia de Ensemble: Stacking

Eu optei pelo Stacking por ser um método que, em vez de uma simples votação ou média, treina um **meta-modelo** para aprender a melhor forma de combinar as previsões dos modelos base. A intuição é criar um "comitê de especialistas" (os modelos base) e um "diretor" (o meta-modelo) que aprende a ponderar a opinião de cada especialista para tomar a decisão final mais acurada.

O processo, gerenciado pelo `StackingClassifier`, funciona da seguinte forma:
1.  As previsões dos modelos base (LGBM e XGB) são geradas através da nossa estratégia de validação cruzada (`StratifiedKFold`).
2.  Essas previsões "fora da amostra" (*Out-of-Fold*) são então usadas como features para treinar o meta-modelo. Eu utilizei o método `predict_proba`, que alimenta o meta-modelo com as probabilidades previstas, um sinal muito mais rico e informativo do que a classe final (0 ou 1).
3.  O meta-modelo aprende a mapear essas probabilidades de entrada para a previsão final.

#### 6.2. Componentes do Modelo de Stacking

* **Modelos Base:** Utilizei as duas versões dos modelos de gradient boosting, com seus hiperparâmetros otimizados pela busca Bayesiana do Optuna. Para maximizar a **diversidade** do ensemble, eu configurei cada modelo com um `random_state` diferente, sendo ambos valores primos e da sequência de Fibonacci, garantindo que a aleatoriedade interna de cada um gerasse modelos com "perspectivas" ligeiramente distintas.

* **Meta-Modelo (`final_estimator`):** Para o meta-modelo, eu escolhi a `LogisticRegression`. Utilizar um modelo linear simples e robusto nesta etapa é uma prática recomendada, pois ele aprende a combinação linear ótima das previsões dos modelos base sem o risco de se sobreajustar a elas, o que melhora a capacidade de generalização do ensemble final.

#### 6.3. Treinamento e Geração da Submissão

O `StackingClassifier` foi integrado a um `Pipeline` final junto com o `preprocessor`. Este pipeline foi treinado com todos os dados de treino disponíveis, garantindo o máximo aprendizado. Por fim, o pipeline treinado foi utilizado para gerar as previsões no conjunto de teste, resultando no arquivo de submissão final.

In [ ]:
# Crie instâncias dos modelos com os melhores parâmetros
lgbm_final = LGBMClassifier(**best_params_dict['LGBM'], random_state=13)
xgb_final = XGBClassifier(**best_params_dict['XGB'], random_state=55)

# O meta-modelo pode ser uma Regressão Logística, que é simples e robusta
meta_model = LogisticRegression(random_state=42, n_jobs=-1)

# Defina a lista de estimadores para o Stacking
estimators = [
    ('LGBM', lgbm_final),
    ('XGBoost', xgb_final)
]

# Crie o Stacking Classifier
stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=cv_strategy,
    stack_method='predict_proba'
)

# Crie o pipeline final
stacking_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('stacker', stacking_model)
])

colunas_para_adicionar = ['grau_instrucao', 'qtde_contas_bancarias_especiais']

# Adiciona as colunas do dataframe original para os dataframes X e X_teste
X[colunas_para_adicionar] = df_treino_raw[colunas_para_adicionar]
X_teste[colunas_para_adicionar] = df_teste_raw[colunas_para_adicionar]

# Treine o modelo final com todos os dados de treino
print("\nTreinando o modelo de Stacking final...")
stacking_pipeline.fit(X, y)
print("Treinamento concluído.")

# Faça as predições
final_predictions = stacking_pipeline.predict(X_teste)

# Crie e salve o arquivo de submissão
submission_df = pd.DataFrame({'id_solicitante': ids_teste, 'inadimplente': final_predictions})
submission_df.to_csv('submission_FINAL_classificacao.csv', index=False)

print("\nArquivo 'submission_FINAL_classificacao.csv' criado com sucesso!")

### **7. Conclusão e Resultados**

Este projeto abordou de forma completa o desafio de classificação de risco de crédito, desde a limpeza inicial dos dados até a implementação de um modelo de ensemble avançado. O processo foi iterativo e guiado por uma estratégia de validação robusta, permitindo a construção de uma solução de alta performance.

O resultado final foi um `StackingClassifier` que combinou as forças de dois modelos de gradient boosting (LightGBM e XGBoost), otimizados individualmente.

Os principais aprendizados e fatores de sucesso foram:
* A importância de uma **validação cruzada estratificada** para obter estimativas de performance confiáveis.
* A eficiência da **Otimização Bayesiana (Optuna)** para explorar espaços de busca complexos e encontrar hiperparâmetros de alta qualidade.
* O poder do **Stacking** como técnica de ensemble, que se provou superior aos modelos individuais ao aprender a melhor forma de combinar suas previsões.